In [68]:
import pandas as pd
import numpy as np

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import LSTM, Input, InputLayer, Dropout, Dense, Flatten, Embedding, Add
from tensorflow.keras import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.preprocessing import LabelEncoder
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.optimizers import RMSprop

In [10]:
df = pd.read_csv('./data/kokil dec 6 reprepare/affcon_final_with_linguistics.csv')

In [11]:
df = df.fillna(0)

In [106]:
train, test = train_test_split(df, test_size=0.2)

y_train_deception = to_categorical(train['Input.deception_quadrant_cat'].tolist())
y_train_rapport_c = to_categorical(train['affcon_rapport'].tolist())
y_train_rapport = train['affcon_rapport'].tolist()
y_train_share_information = train['affcon_shareinformation'].tolist()
y_train_reasoning = train['affcon_reasoning'].tolist()
y_train_gamemove = train['affcon_gamemove'].tolist()

y_test_deception = to_categorical(test['Input.deception_quadrant_cat'].tolist())
y_test_rapport_c = to_categorical(test['affcon_rapport'].tolist())
y_test_rapport = test['affcon_rapport'].tolist()
y_test_share_information = test['affcon_shareinformation'].tolist()
y_test_reasoning = test['affcon_reasoning'].tolist()
y_test_gamemove = test['affcon_gamemove'].tolist()

X_train = train.drop(columns=['affcon_gamemove', 'affcon_reasoning',
                              'affcon_rapport', 'affcon_shareinformation', 'Input.deception_quadrant',
                              'Input.deception_quadrant_cat'])

X_test = test.drop(columns=['affcon_gamemove', 'affcon_reasoning',
                              'affcon_rapport', 'affcon_shareinformation', 'Input.deception_quadrant',
                              'Input.deception_quadrant_cat'])

### Individual classifiers are Random Forests

In [83]:
# Game move classifier
clf_gamemove = RandomForestClassifier(max_depth=2, random_state=0)
clf_gamemove.fit(X_train, y_train_gamemove)
y_pred_gamemove = clf_gamemove.predict(X_train)
y_pred_test_gamemove = clf_gamemove.predict(X_test)
print(classification_report(y_pred_gamemove, y_train_gamemove))

              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00         0
         1.0       1.00      0.84      0.91      3210

    accuracy                           0.84      3210
   macro avg       0.50      0.42      0.46      3210
weighted avg       1.00      0.84      0.91      3210



C:\Users\lynne\anaconda3\envs\tf-gpu\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [107]:
# Rapport classifier
clf_rapport = RandomForestClassifier(max_depth=2, random_state=0)
clf_rapport.fit(X_train, y_train_rapport)
y_pred_rapport = clf_rapport.predict(X_train)
y_pred_test_rapport = clf_rapport.predict(X_test)
print(classification_report(y_pred_rapport, y_train_rapport))

              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00         0
         1.0       1.00      0.83      0.91      3210

    accuracy                           0.83      3210
   macro avg       0.50      0.42      0.45      3210
weighted avg       1.00      0.83      0.91      3210



C:\Users\lynne\anaconda3\envs\tf-gpu\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [85]:
# Share Information classifier
clf_shareinfo = RandomForestClassifier(max_depth=2, random_state=0)
clf_shareinfo.fit(X_train, y_train_share_information)
y_pred_shareinfo = clf_shareinfo.predict(X_train)
y_pred_test_shareinfo = clf_shareinfo.predict(X_test)
print(classification_report(y_pred_shareinfo, y_train_share_information))

              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00         0
         1.0       1.00      0.81      0.89      3210

    accuracy                           0.81      3210
   macro avg       0.50      0.40      0.45      3210
weighted avg       1.00      0.81      0.89      3210



C:\Users\lynne\anaconda3\envs\tf-gpu\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [86]:
# Reasoning classifier
clf_reasoning = RandomForestClassifier(max_depth=2, random_state=0)
clf_reasoning.fit(X_train, y_train_reasoning)
y_pred_reasoning = clf_reasoning.predict(X_train)
y_pred_test_reasoning = clf_reasoning.predict(X_test)
print(classification_report(y_pred_reasoning, y_train_reasoning))

              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00         0
         1.0       1.00      0.80      0.89      3210

    accuracy                           0.80      3210
   macro avg       0.50      0.40      0.44      3210
weighted avg       1.00      0.80      0.89      3210



C:\Users\lynne\anaconda3\envs\tf-gpu\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [109]:
pred_df_arr = []
for i in range(0, len(y_pred_gamemove)):
    pred_obj = {}
    pred_obj['gamemove'] = y_pred_gamemove[i]
    pred_obj['reasoning'] = y_pred_reasoning[i]
    pred_obj['shareinfo'] = y_pred_shareinfo[i]
    pred_obj['rapport'] = y_pred_rapport[i]
    
    pred_df_arr.append(pred_obj)
    
pred_df = pd.DataFrame(pred_df_arr)

In [110]:
pred_test_df_arr = []
for i in range(0, len(y_pred_test_gamemove)):
    pred_obj = {}
    pred_obj['gamemove'] = y_pred_test_gamemove[i]
    pred_obj['reasoning'] = y_pred_test_reasoning[i]
    pred_obj['shareinfo'] = y_pred_test_shareinfo[i]
    pred_obj['rapport'] = y_pred_test_rapport[i]
    
    pred_test_df_arr.append(pred_obj)
    
pred_test_df = pd.DataFrame(pred_test_df_arr)

In [51]:
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

callback = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', min_delta=0, patience=0, verbose=0, mode='auto',
    baseline=None, restore_best_weights=False
)

In [112]:
# Combined model

inputB = Input(shape=(pred_df.shape[1],))
c = Dense(2, activation='relu')(inputB)
c = Dense(4, activation='relu')(c)
c = Dense(2, activation='linear')(c)
m = Model(inputs=inputB, outputs=c)

m.compile(loss='binary_crossentropy', optimizer='adam', 
                      metrics=['acc',f1_m, precision_m,recall_m])
history = m.fit(x=pred_df, y=y_train_deception, epochs=32, 
                    batch_size=64, 
                    validation_data=(pred_test_df,y_test_deception), callbacks=[callback])

Epoch 1/32
51/51 [==============================] - 0s 8ms/step - loss: 7.7125 - acc: 0.0458 - f1_m: 0.0000e+00 - precision_m: 0.0000e+00 - recall_m: 0.0000e+00 - val_loss: 7.7125 - val_acc: 0.0523 - val_f1_m: 0.0000e+00 - val_precision_m: 0.0000e+00 - val_recall_m: 0.0000e+00
Epoch 2/32
51/51 [==============================] - 0s 4ms/step - loss: 7.7125 - acc: 0.0458 - f1_m: 0.0000e+00 - precision_m: 0.0000e+00 - recall_m: 0.0000e+00 - val_loss: 7.7125 - val_acc: 0.0523 - val_f1_m: 0.0000e+00 - val_precision_m: 0.0000e+00 - val_recall_m: 0.0000e+00


### Individual Classifiers are MLP

In [113]:
y_train_deception = to_categorical(train['Input.deception_quadrant_cat'].tolist())
y_train_rapport = to_categorical(train['affcon_rapport'].tolist())
y_train_share_information = to_categorical(train['affcon_shareinformation'].tolist())
y_train_reasoning = to_categorical(train['affcon_reasoning'].tolist())
y_train_gamemove = to_categorical(train['affcon_gamemove'].tolist())

y_test_deception = to_categorical(test['Input.deception_quadrant_cat'].tolist())
y_test_rapport = to_categorical(test['affcon_rapport'].tolist())
y_test_share_information = to_categorical(test['affcon_shareinformation'].tolist())
y_test_reasoning = to_categorical(test['affcon_reasoning'].tolist())
y_test_gamemove = to_categorical(test['affcon_gamemove'].tolist())

X_train = train.drop(columns=['affcon_gamemove', 'affcon_reasoning',
                              'affcon_rapport', 'affcon_shareinformation', 'Input.deception_quadrant',
                              'Input.deception_quadrant_cat'])

X_test = test.drop(columns=['affcon_gamemove', 'affcon_reasoning',
                              'affcon_rapport', 'affcon_shareinformation', 'Input.deception_quadrant',
                              'Input.deception_quadrant_cat'])

In [114]:
# Game move model
inputB = Input(shape=(X_train.shape[1],))
c = Dense(2, activation='relu')(inputB)
c = Dense(4, activation='relu')(c)
c = Dense(2, activation='linear')(c)
gamemove_model = Model(inputs=inputB, outputs=c)

gamemove_model.compile(loss='binary_crossentropy', optimizer='adam', 
                      metrics=['acc',f1_m,precision_m, recall_m])
history = gamemove_model.fit(x=X_train, y=y_train_gamemove, epochs=32, 
                    batch_size=64, 
                    validation_split=0.2, callbacks=[callback])
loss, accuracy, f1_score, precision, recall = gamemove_model.evaluate(X_test, y_test_gamemove, verbose=0)
print(precision, recall, f1_score)

y_pred_gamemove = gamemove_model.predict(X_train)
y_pred_test_gamemove = gamemove_model.predict(X_test)

Epoch 1/32
41/41 [==============================] - 0s 8ms/step - loss: 4.4695 - acc: 0.8357 - f1_m: 0.5196 - precision_m: 0.5271 - recall_m: 0.5343 - val_loss: 3.3540 - val_acc: 0.8489 - val_f1_m: 0.4062 - val_precision_m: 0.5493 - val_recall_m: 0.3281
Epoch 2/32
41/41 [==============================] - 0s 4ms/step - loss: 3.0580 - acc: 0.8431 - f1_m: 0.3630 - precision_m: 0.5515 - recall_m: 0.2763 - val_loss: 3.0895 - val_acc: 0.8489 - val_f1_m: 0.4206 - val_precision_m: 0.5631 - val_recall_m: 0.3409
Epoch 3/32
41/41 [==============================] - 0s 4ms/step - loss: 2.7283 - acc: 0.8431 - f1_m: 0.2841 - precision_m: 0.5674 - recall_m: 0.2069 - val_loss: 2.3662 - val_acc: 0.8474 - val_f1_m: 0.1205 - val_precision_m: 0.5567 - val_recall_m: 0.0682
Epoch 4/32
41/41 [==============================] - 0s 4ms/step - loss: 2.4846 - acc: 0.8438 - f1_m: 0.2330 - precision_m: 0.6070 - recall_m: 0.1521 - val_loss: 2.5463 - val_acc: 0.8489 - val_f1_m: 0.3652 - val_precision_m: 0.6157 - val_r

In [ ]:
# Reasoning model
inputB = Input(shape=(X_train.shape[1],))
c = Dense(2, activation='relu')(inputB)
c = Dense(4, activation='relu')(c)
c = Dense(2, activation='linear')(c)
gamemove_model = Model(inputs=inputB, outputs=c)

gamemove_model.compile(loss='binary_crossentropy', optimizer='adam', 
                      metrics=['acc',f1_m,precision_m, recall_m])
history = gamemove_model.fit(x=X_train, y=y_train_gamemove, epochs=32, 
                    batch_size=64, 
                    validation_split=0.2, callbacks=[callback])
loss, accuracy, f1_score, precision, recall = gamemove_model.evaluate(X_test, y_test_gamemove, verbose=0)
print(precision, recall, f1_score)

y_pred_gamemove = gamemove_model.predict(X_train)
y_pred_test_gamemove = gamemove_model.predict(X_test)

### Individual classifiers are LSTMs

In [55]:
train, test = train_test_split(df, test_size=0.2)

y_train_deception = train['Input.deception_quadrant_cat'].tolist()
y_train_rapport = train['affcon_rapport'].tolist()
y_train_share_information = train['affcon_shareinformation'].tolist()
y_train_reasoning = train['affcon_reasoning'].tolist()
y_train_gamemove = train['affcon_gamemove'].tolist()

y_test_deception = test['Input.deception_quadrant_cat'].tolist()
y_test_rapport = test['affcon_rapport'].tolist()
y_test_share_information = test['affcon_shareinformation'].tolist()
y_test_reasoning = test['affcon_reasoning'].tolist()
y_test_gamemove = test['affcon_gamemove'].tolist()

X_train_col = train['Input.full_text']

X_test_col = test['Input.full_text']

In [61]:
le = LabelEncoder()

y_train_deception = le.fit_transform(y_train_deception)
y_train_deception = y_train_deception.reshape(-1,1)

y_train_rapport = le.fit_transform(y_train_rapport)
y_train_rapport = y_train_rapport.reshape(-1,1)

y_train_share_information = le.fit_transform(y_train_share_information)
y_train_share_information = y_train_share_information.reshape(-1,1)

y_train_reasoning = le.fit_transform(y_train_reasoning)
y_train_reasoning = y_train_reasoning.reshape(-1,1)

y_train_gamemove = le.fit_transform(y_train_gamemove)
y_train_gamemove = y_train_gamemove.reshape(-1,1)

y_test_rapport = le.fit_transform(y_test_rapport)
y_test_rapport = y_test_rapport.reshape(-1,1)

y_test_share_information = le.fit_transform(y_test_share_information)
y_test_share_information = y_test_share_information.reshape(-1,1)

y_test_reasoning = le.fit_transform(y_test_reasoning)
y_test_reasoning = y_test_reasoning.reshape(-1,1)

y_test_gamemove = le.fit_transform(y_test_gamemove)
y_test_gamemove = y_test_gamemove.reshape(-1,1)

In [58]:
max_words = 1000
max_len = 220

tok = Tokenizer(num_words=max_words)

tok.fit_on_texts(X_train_col)
X_train_sequences = tok.texts_to_sequences(X_train_col)
X_train = pad_sequences(X_train_sequences, maxlen=max_len)

tok.fit_on_texts(X_test_col)
X_test_sequences = tok.texts_to_sequences(X_test_col)
X_test = pad_sequences(X_test_sequences, maxlen=max_len)

In [48]:
early_stop = EarlyStopping(monitor='val_loss',min_delta=0.00001)
def create_lstm():
    Inp = Input(name='inputs', shape=[max_len])
    x = Embedding(max_words, 50, input_length=max_len)(Inp)
    x = LSTM(64, name='LSTM_01')(x)
    x = Dropout(0.5, name='Dropout')(x)
    x = Dense(128, activation='relu',name='Dense_01')(x)
    # x = Dropout(0.5,name='Dropout')(x)
    out = Dense(1,activation='sigmoid', name='output')(x)
    model = Model(inputs=Inp, outputs=out)
    
    return model

In [60]:
# Rapport model
rapport_model = create_lstm()
rapport_model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])
rapport_model.fit(X_train,y_train_rapport,
          batch_size=128,
          epochs=15,
          validation_data=(X_test, y_test_rapport), callbacks=[early_stop])

Epoch 1/15
26/26 [==============================] - 1s 52ms/step - loss: 0.5210 - accuracy: 0.8184 - val_loss: 0.4999 - val_accuracy: 0.8356
Epoch 2/15
26/26 [==============================] - 1s 36ms/step - loss: 0.4668 - accuracy: 0.8265 - val_loss: 0.4528 - val_accuracy: 0.8356
Epoch 3/15
26/26 [==============================] - 1s 36ms/step - loss: 0.4661 - accuracy: 0.8265 - val_loss: 0.4686 - val_accuracy: 0.8356


In [62]:
# Game move model
gamemove_model = create_lstm()
gamemove_model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])
gamemove_model.fit(X_train,y_train_gamemove,
          batch_size=128,
          epochs=15,
          validation_data=(X_test, y_test_gamemove), callbacks=[early_stop])

Epoch 1/15
26/26 [==============================] - 1s 48ms/step - loss: 0.4875 - accuracy: 0.8174 - val_loss: 0.4051 - val_accuracy: 0.8605
Epoch 2/15
26/26 [==============================] - 1s 33ms/step - loss: 0.4462 - accuracy: 0.8389 - val_loss: 0.4059 - val_accuracy: 0.8605


In [63]:
# Reasoning model
reasoning_model = create_lstm()
reasoning_model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])
reasoning_model.fit(X_train,y_train_reasoning,
          batch_size=128,
          epochs=15,
          validation_data=(X_test, y_test_reasoning), callbacks=[early_stop])

Epoch 1/15
26/26 [==============================] - 1s 49ms/step - loss: 0.5367 - accuracy: 0.7947 - val_loss: 0.4990 - val_accuracy: 0.8095
Epoch 2/15
26/26 [==============================] - 1s 33ms/step - loss: 0.5058 - accuracy: 0.8009 - val_loss: 0.4884 - val_accuracy: 0.8095
Epoch 3/15
26/26 [==============================] - 1s 33ms/step - loss: 0.5004 - accuracy: 0.8009 - val_loss: 0.4916 - val_accuracy: 0.8095


In [65]:
# Share Information model
shareinfo_model = create_lstm()
shareinfo_model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])
shareinfo_model.fit(X_train,y_train_share_information,
          batch_size=128,
          epochs=15,
          validation_data=(X_test, y_test_share_information), callbacks=[early_stop])

Epoch 1/15
26/26 [==============================] - 2s 61ms/step - loss: 0.5280 - accuracy: 0.7935 - val_loss: 0.4761 - val_accuracy: 0.8306
Epoch 2/15
26/26 [==============================] - 1s 33ms/step - loss: 0.5030 - accuracy: 0.8044 - val_loss: 0.5416 - val_accuracy: 0.8306


In [67]:
# Joint Model
commonInput = Input(shape=[max_len])

out1 = gamemove_model(commonInput)    
out2 = reasoning_model(commonInput)   
out3 = shareinfo_model(commonInput)
out4 = rapport_model(commonInput)
mergedOut = Add()([out1,out2,out3,out4])

mergedOut = Flatten()(mergedOut)    
mergedOut = Dense(256, activation='relu')(mergedOut)
mergedOut = Dropout(.5)(mergedOut)
mergedOut = Dense(128, activation='relu')(mergedOut)
mergedOut = Dropout(.35)(mergedOut)
mergedOut = tf.keras.layers.Dense(2, activation='softmax')(mergedOut)  #Cuz binary

mergedModel = tf.keras.Model(commonInput, mergedOut)
mergedModel.compile(loss='binary_crossentropy', optimizer=RMSprop(), metrics=['accuracy'])
shareinfo_model.fit(X_train,y_train_deception,
          batch_size=128,
          epochs=15,
          validation_data=(X_test, y_test_deception), callbacks=[early_stop])

NameError: name 'max_length' is not defined